<a href="https://colab.research.google.com/github/uresha1995/Research-Methodology/blob/main/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --no-cache-dir transformers==4.38.2 datasets==2.18.0 evaluate==0.4.1

In [2]:
#Import libraries

from datasets import load_dataset
from transformers import BertTokenizerFast, BertForSequenceClassification, TrainingArguments
#from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader
#import evaluate
import torch
from tqdm import tqdm
import re
import random

In [3]:
#Download dataset

dataset = load_dataset("imdb")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
#Print  random reviews

sample_texts = [dataset["train"][i]["text"] for i in random.sample(range(25000), 5)]

for i, text in enumerate(sample_texts):
    print(f"--- Sample {i+1} ---\n{text[:500]}\n")

--- Sample 1 ---
At the time, "My Left Foot" was the little movie that could. It was hugely popular, and everyone applauded the fact that such a small, independent film could make it all the way to the Oscars.<br /><br />Since then, movies like "My Left Foot" are a dime a dozen, so it might be hard in retrospect to understand what all the fuss was about. It's certainly a well made and competent film, but it's clear that the bulk of its success rests on the shoulders of Daniel Day-Lewis, who immerses himself in t

--- Sample 2 ---
This film on paper looked like it could possibly be good, after watching though i realised that this film was completely terrible!! The plot has no meaning, and i think i counted the best part of 5000 cut scenes each one making the film more annoying boring and ridiculous. I watched this late night pitch black no noise at all just to add to the SCARINESS of it but the truth is the only thing that scared me was the music, what they would call tragic music, they

Preprocessing

In [5]:
#Remove empty and short reviews

def rem_empty(example):
    return len(example["text"].strip()) > 20

In [7]:
#Remove HTML break tags and replace with space

def clean_text(example):
    text = example["text"]
    text = re.sub(r"<br\s*/?>", " ", text)
    example["text"] = text.strip()
    return example

In [6]:
dataset["train"] = dataset["train"].filter(rem_empty)
dataset["test"] = dataset["test"].filter(rem_empty)
dataset = dataset.map(clean_text)

Tokenization using BertTokenizerFast

In [16]:
#Making subset for fast the training

train_data = dataset["train"].shuffle(seed=42).select(range(2000))
test_data = dataset["test"].shuffle(seed=42).select(range(1000))

In [15]:
#Define tokenization function

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

def tokenize_function(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",   # ensures equal-length batches
        max_length=128
    )

In [17]:
#Apply tokenization

#tokenized_train = small_train.map(tokenize_function, batched=True)
#tokenized_test = small_test.map(tokenize_function, batched=True)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [10]:
#Convert to torch tensors

train_data.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_data.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

In [18]:
from transformers import BertTokenizer, BertForSequenceClassification

In [ ]:
#DataLoader

train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
test_loader = DataLoader(test_data, batch_size=16)

In [19]:
#Load model

#tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
#model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2).to(device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
#Apply tokenization

#train_data = train_data.map(tokenize_function, batched=True)
#test_data = test_data.map(tokenize_function, batched=True)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [21]:
#Set device

#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [22]:
#Define training arguments

#training_args = TrainingArguments(
#    output_dir="./results",
 #   evaluation_strategy="epoch",
  #  save_strategy="epoch",
   # logging_dir="./logs",
    #logging_steps=50,
    #learning_rate=2e-5,
    #per_device_train_batch_size=16,
    #per_device_eval_batch_size=16,
    #num_train_epochs=2,
    #weight_decay=0.01,
    #load_best_model_at_end=True,
    #report_to="none"
#)

In [23]:
#Convert tokenized dataset into torch tensors

##tokenized_train.set_format("torch", columns=["input_ids", "attention_mask", "label"])
tokenized_test.set_format("torch", columns=["input_ids", "attention_mask", "label"])


In [24]:
#Prepare dataLoader

#train_loader = DataLoader(tokenized_train, batch_size=16, shuffle=True)
#test_loader = DataLoader(tokenized_test, batch_size=16)

In [25]:
#Move model device

#from transformers import BertForSequenceClassification

#model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [26]:
#Optimizer

from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [27]:
from tqdm import tqdm

In [ ]:
#Trainin the loop

#epochs = 2
#for epoch in range(epochs):
   # model.train()
   # total_loss = 0
   # for batch in tqdm(train_loader):
   #     batch = {k: v.to(device) for k, v in batch.items()}
   #     if 'label' in batch:
    #        batch['labels'] = batch.pop('label')  # required by Hugging Face model
     #   outputs = model(**batch)
     #   loss = outputs.loss
     #   loss.backward()
      #  optimizer.step()
      #  optimizer.zero_grad()
      #  total_loss += loss.item()
      #  torch.cuda.empty_cache()
  #  print(f"Epoch {epoch + 1} - Training Loss: {total_loss:.4f}")

In [ ]:
epochs = 2
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader):
        batch = {k: v.to(device) for k, v in batch.items()}
        batch['labels'] = batch.pop('label')  # Ensure label is correctly named
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1} - Training Loss: {total_loss:.4f}")

100%|██████████| 125/125 [51:28<00:00, 24.71s/it]


Epoch 1 - Training Loss: 66.0091


 70%|███████   | 88/125 [37:23<15:02, 24.38s/it]

In [ ]:
from torch.utils.data import DataLoader
import evaluate

#Create test DataLoader
test_loader = DataLoader(test_data, batch_size=16)

#Evaluation
model.eval()
accuracy = evaluate.load("accuracy")
predictions, labels = [], []

with torch.no_grad():
    for batch in test_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        if 'label' in batch:
            batch['labels'] = batch.pop('label')
        outputs = model(**batch)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)
        predictions.extend(preds.cpu().numpy())
        labels.extend(batch["labels"].cpu().numpy())

In [ ]:
#Final accuracy score

result = accuracy.compute(predictions=predictions, references=labels)
print("Test Accuracy:", result)